# tf.function性能优化

尽管eager执行更简洁，但是Graph模式却是性能更高，为了减少这个性能gap，TensorFlow 2.0引入了tf.function
> 在TensorFlow 2.0，我们一般不会直接使用 `tf.autograph`，因为eager执行下效率没有提升。要真正达到Graph模式下的效率，要依赖 ```tf.function``` 这个更强大的利器。

- [1 计算性能更优](#1.计算性能更优)
- [2 调用便捷](#2.调用便捷)
- [3 多态性polymorphism](#3.多态性polymorphism)
- [4 指定输入参数类型 input_signature](#4.指定输入参数类型input_signature)
- [5 指定参数autograph](#5.指定参数autograph)
- [6 应用到类方法中](#6.应用到类方法中)
- [7 tf.print看具体数值](#7.tf.print看具体数值)
- [8 dubug时设置tf.config.experimental_run_functions_eagerly=True](#8.dubug时设置tf.config.experimental_run_functions_eagerly=True)

# 1.计算性能更优
简单来说，就是tf.function可以将一个func中的TensorFlow操作构建为一个Graph，这样在调用时是执行这个Graph，这样计算性能更优.


In [1]:
'''
被tf.function装饰的函数将以Graph模式执行，可以把它想象一个封装了Graph的TF op，直接调用它也会立即得到Tensor结果，
但是其内部是高效执行的。
我们在内部打印Tensor时，eager执行会直接打印Tensor的值，
而Graph模式打印的是Tensor句柄，其无法调用numpy方法取出值，这和TF1.x的Graph模式是一致的。
'''

import tensorflow as tf
import timeit

def f(x, y):
#     print(x, y)
    return tf.reduce_mean(tf.multiply(x ** 2, 3) + y)
g = tf.function(f)
x = tf.constant([[2.0, 3.0]])
y = tf.constant([[3.0, -2.0]])
# `f` and `g` will return the same value, but `g` will be executed as a TensorFlow graph.

print(f(x, y).numpy() == g(x, y).numpy())    #区别在于：Graph模式打印的是Tensor句柄，其无法调用numpy方法取出值
# assert f(x, y).numpy() == g(x, y).numpy()

print("eager model:", timeit.timeit(lambda: f(x,y), number=100))   
print("graph model:", timeit.timeit(lambda: g(x,y), number=100))

# 通过时间对比，如果注释掉函数中的 print(x,y) ，则会运行时间相差不大，这就代表eager模式的取值使得性能降低了，如果存在复杂运算，这种优化效果会更加明显。

True
eager model: 0.027376342564821243
graph model: 0.024105901829898357


In [2]:
"""
由于tf.function装饰的函数是Graph执行，其执行速度一般要比eager模式要快.
当Graph包含很多小操作时差距更明显，可以比较下卷积和LSTM的性能差距
"""

conv_layer = tf.keras.layers.Conv2D(100, 3)
@tf.function
def conv_fn(image):
  return conv_layer(image)
image = tf.zeros([1, 200, 200, 100])
# warm up
conv_layer(image); conv_fn(image)
print("Eager conv:", timeit.timeit(lambda: conv_layer(image), number=10))
print("Function conv:", timeit.timeit(lambda: conv_fn(image), number=10))
# 单纯的卷积差距不是很大

print("-------------------------------------------------------------")

lstm_cell = tf.keras.layers.LSTMCell(10)
@tf.function
def lstm_fn(input, state):
  return lstm_cell(input, state)
input = tf.zeros([10, 10])
state = [tf.zeros([10, 10])] * 2
# warm up
lstm_cell(input, state); lstm_fn(input, state)
print("eager lstm:", timeit.timeit(lambda: lstm_cell(input, state), number=10))
print("function lstm:", timeit.timeit(lambda: lstm_fn(input, state), number=10))
# 对于LSTM比较heavy的计算，Graph执行要快很多

Eager conv: 0.7734643267467618
Function conv: 0.7347989389672875
-------------------------------------------------------------
eager lstm: 0.05459785647690296
function lstm: 0.004435085691511631


# 2.调用便捷

TF 1.X的例子：

```python
x = tf.placeholder(tf.float32)
y = tf.square(x)
z = tf.add(x, y)
sess = tf.Session()
z0 = sess.run([z], feed_dict={x: 2.})        # 6.0
z1 = sess.run([z], feed_dict={x: 2., y: 2.}) # 4.0
```

尽管上面只定义了一个graph，但是两次不同的sess执行（运行时）其实是执行两个不同的程序或者说subgraph,即下面改成之后的两个函数。

In [3]:
"""
这里我们将两个不同的subgraph封装到了两个python函数中。
更进一步地，我们可以不再需要Session，当执行这两个函数时，直接调用对应的计算图就可以，这就是tf.function的功效
"""

@tf.function
def compute_z1(x, y):
  return tf.add(x, y)
@tf.function
def compute_z0(x):
  return compute_z1(x, tf.square(x))
z0 = compute_z0(2.)
z1 = compute_z1(2., 2.)
print(z0)
print(z1)

tf.Tensor(6.0, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)


# 3.多态性polymorphism

虽然函数内部定义了一系列的操作，但是对于不同的输入，是需要不同的计算图。如函数的输入Tensor的shape或者dtype不同，那么计算图是不同的，好在tf.function支持这种多态性（polymorphism）

In [4]:
# Functions are polymorphic
@tf.function
def double(a):
  print("Tracing with", a)
  return a + a
print(double(tf.constant(1)))
print(double(tf.constant(1.1)))
print(double(tf.constant([1, 2])))    

#当输入tensor的shape或者类型发生变化，打印的东西也是相应改变。所以，它们的计算图（静态的）并不一样。tf.function这种多态特性其实是背后追踪了（tracing）不同的计算图。

Tracing with Tensor("a:0", shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
Tracing with Tensor("a:0", shape=(), dtype=float32)
tf.Tensor(2.2, shape=(), dtype=float32)
Tracing with Tensor("a:0", shape=(2,), dtype=int32)
tf.Tensor([2 4], shape=(2,), dtype=int32)


# 4.指定输入参数类型input_signature

tf.function提供了`input_signature`，这个参数采用tf.TensorSpec指定了输入到函数的Tensor的shape和dtypes

```python
@tf.function(input_signature=[tf.TensorSpec(shape=None, dtype=tf.float32)])
def f(x):
    return tf.add(x, 1.)
print(f(tf.constant(1.0)))  # tf.Tensor(2.0, shape=(), dtype=float32)
print(f(tf.constant([1.0,]))) # tf.Tensor([2.], shape=(1,), dtype=float32)
print(f(tf.constant([1])))  # ValueError: Python inputs incompatible with input_signature
```
此时，输入Tensor的dtype必须是float32，但是shape不限制，当类型不匹配时会出错。

# 5.指定参数autograph

tf.function的另外一个参数是`autograph`，默认是True，**意思是在构建Graph时将自动使用AutoGraph，这样你可以在函数内部使用Python原生的条件判断以及循环语句，因为它们会被tf.cond和tf.while_loop转化为Graph代码。**注意的一点是判断分支和循环必须依赖于Tensors才会被转化，当autograph为False时，如果存在判断分支和循环必须依赖于Tensors的情况将会出错。
> 大部分情况，我们还是默认开启autograph

In [5]:
def sum_even(items):
  s = 0
  for c in items:
    if c % 2 > 0:
      continue
    s += c
  return s
sum_even_autograph_on = tf.function(sum_even, autograph=True)
sum_even_autograph_off = tf.function(sum_even, autograph=False)
x = tf.constant([10, 12, 15, 20])
sum_even(x) # OK 
sum_even_autograph_on(x) # OK 比前面的eager模式快
sum_even_autograph_off(x) # error

TypeError: Tensor objects are only iterable when eager execution is enabled. To iterate over this tensor use tf.map_fn.

# 6.应用到类方法中
`tf.function`可以应用到类方法中，并且可以引用`tf.Variable`

这个特性可以应用到`tf.Keras`的模型构建中。下面这个例子还有一点，就是可以在`function`中使用`tf.assign`这类具有副作用（改变Variable的值）的操作，这对于模型训练比较重要。

In [6]:
class ScalarModel(object):
  def __init__(self):
    self.v = tf.Variable(0)
  @tf.function
  def increment(self, amount):
    self.v.assign_add(amount)

model1 = ScalarModel()
model1.increment(tf.constant(3))
assert int(model1.v) == 3
model1.increment(tf.constant(4))
assert int(model1.v) == 7

model2 = ScalarModel()  # model1和model2 拥有不同变量
model2.increment(tf.constant(5))
assert int(model2.v) == 5

# 7.tf.print看具体数值

python原生的print函数只会在构建Graph时打印一次Tensor句柄。如果想要打印Tensor的具体值，要使用tf.print


In [7]:
@tf.function
def print_element(items):
    for c in items:
      tf.print(c)
x = tf.constant([1, 5, 6, 8, 3])
print_element(x)

1
5
6
8
3


# 8.dubug时设置tf.config.experimental_run_functions_eagerly=True

In [2]:
@tf.function
def f(x):
  if x > 0:
    import pdb
    pdb.set_trace()
    x = x + 1
  return x

tf.config.experimental_run_functions_eagerly(True)
f(tf.constant(1))

> <ipython-input-2-26eaebb70cfe>(6)f()
-> x = x + 1


(Pdb)  x


<tf.Tensor: id=828, shape=(), dtype=int32, numpy=1>


(Pdb)  tf.print(x)


1


(Pdb)  q


BdbQuit: 